# Marker Determination

In [ ]:
import sklearn as sk
import anndata as ad
import scanpy as sc 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import joblib

sc.settings.n_jobs = -1

os.chdir("/project/hipaa_ycheng11lab/atlas/CAMR2024/")

In [ ]:
adata = ad.read_h5ad('data/camr_modeling_input.h5ad')
gene_names = adata.var["feature_name"].astype(str)
if not os.path.isfile('data/CAMR_genes.csv'):
    adata.var["feature_name"].to_csv('data/CAMR_genes.csv', index=False)
adata

## Major Class Markers

In [ ]:
top_features_log_reg = pd.read_csv('spreadsheets/ovr_top_20_genes_by_cell_type_reproduction.csv')
# Only positive features, negative markers are less useful
top_features_log_reg_pos = top_features_log_reg[top_features_log_reg['Coefficient'] > 0]
top_features_log_reg_pos.index = top_features_log_reg_pos.Gene
top_features_log_reg_pos

In [ ]:
# Raw average expression
highly_variable = adata.raw.var['feature_name'].isin(gene_names.tolist())
if not os.path.isfile('spreadsheets/raw_mean_variable_genes.csv'):
    raw_feature_expression_pd = pd.DataFrame(adata.raw.X[:, highly_variable].toarray(), columns = gene_names.tolist())
    raw_feature_expression_pd["majorclass"] = adata.obs["majorclass"].tolist()
    raw_feature_expression_pd_mean = raw_feature_expression_pd.groupby("majorclass").agg("mean")
    del raw_feature_expression_pd
    raw_feature_expression_pd_mean.write_csv('spreadsheets/raw_mean_variable_genes.csv')
else:
    raw_feature_expression_pd_mean = pd.read_csv('spreadsheets/raw_mean_variable_genes.csv', index_col=0)
raw_feature_expression_pd_mean

In [ ]:
# Filter based on innate features of the gene itself
in_regression = adata.var["feature_name"].astype(str).isin(top_features_log_reg_pos["Gene"])
long_enough = adata.var["feature_length"].astype(int) >= 960 # It's a conservative filter

keep_genes = long_enough & in_regression
kept_gene_names = gene_names[keep_genes.tolist()].tolist()
print(len(kept_gene_names), kept_gene_names) # 216 genes

In [ ]:
# Filter based on the filtering criteria
# adata.obs.library_platform.unique() shows that this is mix of 4 chemistries, so use strictest?

# count_limit = 0.1 # Absolute detection limit
count_lowcluster = 4
count_highcluster = 100

detectable_genes = (raw_feature_expression_pd_mean >= count_lowcluster).sum(axis=0) >= 1
optical_crowding_genes = (raw_feature_expression_pd_mean > count_highcluster).sum(axis=0) > 0

is_expression_candidate = detectable_genes & (~optical_crowding_genes)

expression_candidates = gene_names[is_expression_candidate.tolist()].tolist()
print(len(expression_candidates), expression_candidates) # 428 genes

In [ ]:
# Combine gene metadata filtering and gene expression filtering together
final_candidates = np.intersect1d(expression_candidates, kept_gene_names)
print(len(final_candidates), final_candidates) # 172 genes

In [ ]:
# Save all genes that passed these thresholds
# Sensitive to refer to the fact that we are not filtering on specificity, not checking off-targets
ordered_features = top_features_log_reg_pos.loc[final_candidates.tolist()].sort_values('Cell Type')
ordered_features.to_csv('spreadsheets/ovr_top_filtered_genes_majorclass_coefficients_sensitive.csv')
ordered_features

In [ ]:
sc.pl.dotplot(adata,
              var_names = ordered_features.index,
              gene_symbols="feature_name",
              groupby = 'majorclass',
              categories_order = adata.obs["majorclass"].cat.categories.sort_values(),
              vmax = count_lowcluster,
              vmin = count_lowcluster / 2,
              save = f"mouseRetina_filteredCounts." +
                     f"{count_lowcluster}-{count_highcluster}_" +
                     f"sensitive.pdf")

In [ ]:
# Average normalized expression for nice dotplots
if not os.path.isfile('spreadsheets/norm_mean_variable_genes.csv'):
    feature_expression_pd = pd.DataFrame(adata.X.toarray(), columns = gene_names.tolist()) # Only variable genes in 
    feature_expression_pd["majorclass"] = adata.obs["majorclass"].tolist()
    feature_expression_pd_mean = feature_expression_pd.groupby("majorclass").agg("mean")
    del feature_expression_pd
    feature_expression_pd_mean.to_csv('spreadsheets/norm_mean_variable_genes.csv')
else:
    feature_expression_pd_mean = pd.read_csv('spreadsheets/norm_mean_variable_genes.csv', index_col=0)
raw_feature_expression_pd_mean[ordered_features.index.unique().tolist()].to_csv('spreadsheets/ovr_top_filtered_genes_majorclass_rawMeanExpression_sensitive.csv')
feature_expression_pd_mean[ordered_features.index.unique().tolist()].to_csv('spreadsheets/ovr_top_filtered_genes_majorclass_normMeanExpression_sensitive.csv')


In [ ]:
# Cleanup
del top_features_log_reg, top_features_log_reg_pos, raw_feature_expression_pd_mean, in_regression, long_enough, keep_genes, kept_gene_names, detectable_genes, optical_crowding_genes, is_expression_candidate, expression_candidates, final_candidates, ordered_features, feature_expression_pd_mean 

## Subtype Markers

In [ ]:
top_features_log_reg_sub = pd.read_csv('spreadsheets/ovr_top_20_genes_by_sub_cell_type_reproduction.csv')
top_features_log_reg_pos_sub = top_features_log_reg_sub[top_features_log_reg_sub['Coefficient'] > 0]
top_features_log_reg_pos_sub.index = top_features_log_reg_pos_sub.Gene
top_features_log_reg_pos_sub

In [ ]:
# Raw average expression
if not os.path.isfile('spreadsheets/raw_subtype_mean_variable_genes.csv'):
    raw_feature_expression_sub = pd.DataFrame(adata.raw.X[:, highly_variable].toarray(), columns = gene_names.tolist())
    raw_feature_expression_sub["author_cell_type"] = adata.obs["author_cell_type"].tolist()
    raw_feature_expression_sub_mean = raw_feature_expression_sub.groupby("author_cell_type").agg("mean")
    del raw_feature_expression_sub
    raw_feature_expression_sub_mean.write_csv('spreadsheets/raw_subtype_mean_variable_genes.csv')
else:
    raw_feature_expression_sub_mean = pd.read_csv('spreadsheets/raw_subtype_mean_variable_genes.csv', index_col=0)
raw_feature_expression_sub_mean

In [ ]:
# Filter based on innate features of the gene itself
in_regression = adata.var["feature_name"].astype(str).isin(top_features_log_reg_pos_sub["Gene"])
long_enough = adata.var["feature_length"].astype(int) >= 960 # It's a conservative filter

keep_genes = long_enough & in_regression
kept_gene_names = gene_names[keep_genes.tolist()].tolist()
print(len(kept_gene_names), kept_gene_names) # 680 genes

In [ ]:
# Filter based on the filtering criteria
# adata.obs.library_platform.unique() # mix of 4 chemistries...

# count_limit = 0.1 # Absolute detection limit
count_lowcluster = 4
count_highcluster = 100

detectable_genes = (raw_feature_expression_sub_mean >= count_lowcluster).sum(axis=0) >= 1
optical_crowding_genes = (raw_feature_expression_sub_mean > count_highcluster).sum(axis=0) > 0

is_expression_candidate = detectable_genes & (~optical_crowding_genes)

expression_candidates = gene_names[is_expression_candidate.tolist()].tolist()
print(len(expression_candidates), expression_candidates) # 669 genes

In [ ]:
final_candidates = np.intersect1d(expression_candidates, kept_gene_names)
print(len(final_candidates), final_candidates) # 479 genes

In [ ]:
ordered_features_sub = top_features_log_reg_pos_sub.loc[final_candidates.tolist()].sort_values('Cell Type')
ordered_features_sub.to_csv('spreadsheets/ovr_top_filtered_genes_author_cell_type_coefficients_sensitive.csv')
ordered_features_sub

In [ ]:
# Memory cleanup for next section

del top_features_log_reg_sub, raw_feature_expression_sub_mean, in_regression, long_enough, keep_genes, kept_gene_names, detectable_genes, optical_crowding_genes, is_expression_candidate, expression_candidates, final_candidates

adata.raw = None
adata.obs = adata.obs.loc[:, ["author_cell_type"]]
adata.var = adata.var.loc[:, ["gene_symbols", "feature_name"]]

import gc
import ctypes
gc.collect() # Free memory
libc = ctypes.CDLL("libc.so.6") # clearing cache 
libc.malloc_trim(0)

In [ ]:
# Asking for lots of memory

sc.pl.dotplot(adata,
              ordered_features_sub.index.unique(),
              gene_symbols="feature_name",
              groupby = 'author_cell_type',
              vmax = count_lowcluster,
              vmin = count_lowcluster / 2,
              figsize = (40, 2),
              save = f"mouseRetina_author_cell_type_filteredCounts." +
                     f"{count_lowcluster}-{count_highcluster}_" +
                     f"sensitive.png")

In [ ]:
# Average expression
if not os.path.isfile('spreadsheets/norm_subtype_mean_variable_genes.csv'):
    feature_expression_sub = pd.DataFrame(adata.X.toarray(), columns = gene_names.tolist())
    feature_expression_sub["author_cell_type"] = adata.obs["author_cell_type"].tolist()
    feature_expression_sub_mean = feature_expression_sub.groupby("author_cell_type").agg("mean")
    del feature_expression_sub
    feature_expression_sub_mean.to_csv('spreadsheets/norm_subtype_mean_variable_genes.csv')
else:
    feature_expression_sub_mean = pd.read_csv('spreadsheets/norm_subtype_mean_variable_genes.csv', index_col=0)
raw_feature_expression_sub_mean = pd.read_csv('spreadsheets/raw_subtype_mean_variable_genes.csv', index_col=0)

raw_feature_expression_sub_mean[ordered_features_sub.index.unique().tolist()].to_csv('spreadsheets/ovr_top_filtered_genes_author_cell_type_rawMeanExpression_sensitive.csv')
feature_expression_sub_mean[ordered_features_sub.index.unique().tolist()].to_csv('spreadsheets/ovr_top_filtered_genes_author_cell_type_normMeanExpression_sensitive.csv')

In [ ]:
# Cleanup
del adata, raw_feature_expression_sub_mean, feature_expression_sub_mean
adata = ad.read_h5ad('data/camr_modeling_input.h5ad')

In [ ]:
# Will be informative as to how accurate some calculations are...
# Needs to be run as an interactive job :(
if not os.path.isfile('spreadsheets/raw_subtype_sum_variable_genes.csv'):
    raw_feature_expression_sub = pd.DataFrame(adata.raw.X[:, highly_variable].toarray(), columns = gene_names.tolist())
    raw_feature_expression_sub["author_cell_type"] = adata.obs["author_cell_type"].tolist()
    raw_feature_expression_sub_sum = raw_feature_expression_sub.groupby("author_cell_type").agg("sum")
    del raw_feature_expression_sub
    raw_feature_expression_sub_sum.write_csv('spreadsheets/raw_subtype_sum_variable_genes.csv')
else:
    raw_feature_expression_sub_sum = pd.read_csv('spreadsheets/raw_subtype_sum_variable_genes.csv', index_col=0)
feature_expression_pd_sub_sum[ordered_features_sub.index.unique().tolist()].to_csv('spreadsheets/ovr_top_filtered_genes_author_cell_type_rawSumExpression_sensitive.csv')

### Subtype Marker Better Plots

Requirements:

* adata
* final_candidates_ordered
* final_candidates_ordered_sub
* count_lowcluster
* count_highcluster

In [ ]:
# Run this if starting from scratch!

import sklearn as sk
import anndata as ad
import scanpy as sc 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import joblib

sc.settings.n_jobs = -1

count_lowcluster = 4
count_highcluster = 100

adata = ad.read_h5ad('data/camr_modeling_input.h5ad')
adata.var.index = adata.var["feature_name"] # subset on genes instead of booleans

final_candidates_ordered = pd.read_csv('spreadsheets/ovr_top_filtered_genes_majorclass_coefficients_sensitive.csv', index_col = 0)
final_candidates_ordered_sub = pd.read_csv('spreadsheets/ovr_top_filtered_genes_author_cell_type_coefficients_sensitive.csv', index_col = 0)

In [ ]:
majorclass = final_candidates_ordered["Cell Type"].unique()
minorclass = final_candidates_ordered_sub["Cell Type"].unique() # Not necessary
print(majorclass, minorclass)

In [ ]:
subtype_to_type = adata.obs[["author_cell_type", "majorclass"]].groupby("author_cell_type").head(1)
subtype_to_type.groupby("majorclass").agg("count").to_csv('spreadsheets/number_of_subtypes.csv')
subtype_to_type

In [ ]:
# Remove some memory hogs
# del adata
# adata = ad.read_h5ad('data/camr_modeling_input.h5ad')

adata.raw = None
adata.obs = adata.obs.loc[:, ["majorclass", "author_cell_type"]]
adata.var = adata.var.loc[:, ["gene_symbols", "feature_name"]]

adata.var.index = adata.var["feature_name"] # subset on genes instead of booleans
adata.var_names_make_unique()

import gc
import ctypes
gc.collect() # Free memory
libc = ctypes.CDLL("libc.so.6") # clearing cache 
libc.malloc_trim(0)

In [ ]:
for cell in majorclass:

    cell_markers = final_candidates_ordered[final_candidates_ordered == cell].index
    subtypes = subtype_to_type.loc[subtype_to_type.majorclass == cell, "author_cell_type"].tolist()
    subtype_markers = final_candidates_ordered_sub[final_candidates_ordered_sub.isin(subtypes)].index

    markers = cell_markers.tolist() + subtype_markers.tolist()

    # sc.pl.dotplot throws a fit if there are duplicates
    unique_markers = []
    for m in markers:
        if m not in unique_markers:
            unique_markers += [m]

    sc.pl.dotplot(adata[adata.obs.majorclass == cell, unique_markers],
                  unique_markers,
                  gene_symbols="feature_name",
                  groupby = 'author_cell_type',
                  vmax = count_lowcluster * 3,
                  vmin = count_lowcluster - 1,
                  show = False,
                  save = f"mouseRetina_{cell}.author_cell_type_filteredCounts." +
                         f"{count_lowcluster}-{count_highcluster}.pdf")

# Scratch

In [ ]:
sum([g == gene_names.tolist()[i] for i, g in enumerate(adata.raw.var['feature_name'][adata.raw.var['feature_name'].isin(adata.var['feature_name'])])])

In [ ]:
[cat for cat in adata.obs.author_cell_type.cat.categories.tolist()
     if cat not in adata.obs.subclass_label.cat.categories.tolist()]

In [ ]:
adata.obs.columns

In [ ]:
adata.obs.author_cell_type.value_counts() # TODO: Add this as a metadata column